# Example: using a different force field

Here we show how to use MDPOW to calculate partition coefficients using an force field that isn't included in the package. To follow along, install `jupyter` in your `mdpow` environment.

To implement a new force field, you will need:

* `ITP` files for the molecule, the solvents, ions and also the general atom type definitions file (usually just named after the force field itself).
* `MDP` files for the energy minimisation, initial relaxation, NPT ensemble run and free energy calculation.
* Structure files (`.gro` or `.pdb`) for the solute and non-aqueous solvent. If you are using a type of water that does not come bundled with GROMACS, like in this example, you will also need to create an equilibrated box of pure water.

The first thing we'll do is to download the files we need for Martini 3.0.

In [19]:
from pathlib import Path
from typing import Optional

import requests as req
from zipfile import ZipFile


HERE = Path(".")
MARTINI_FF = HERE / "martini.ff"
MARTINI_FF.mkdir(exist_ok=True)

MARTINI_ITP = MARTINI_FF / "forcefield.itp"
MARTINI_IONS = MARTINI_FF / "martini_v3.0.0_ions_v1.itp"
MARTINI_SMALL_MOLS = MARTINI_FF / "martini_v3.0.0_small_molecules_v1.itp"
MARTINI_SOLVENTS = MARTINI_FF / "martini_v3.0.0_solvents_v1.itp"
BENZENE_ITP = MARTINI_SMALL_MOLS

MARTINI_WATER = HERE / "water.gro"
MARTINI_OCTANOL = HERE / "octanol.gro"
MARTINI_BENZENE = HERE / "benzene.pdb"

EM_FILE = HERE / "em.mdp"
EQ_FILE = HERE / "eq.mdp"
RUN_FILE = HERE / "run.mdp"


def download_file(
    url: str, out: Optional[Path] = None, chunk_size: int = 128, overwrite: bool = False
):
    """Utility function to download files."""
    if out is None:
        out = HERE / Path(url).name

    if out.exists() and not overwrite:
        return

    r = req.get(url, stream=True)
    r.raise_for_status()

    with out.open("wb") as f:
        for chunk in r.iter_content(chunk_size=chunk_size):
            f.write(chunk)


DOWNLOADS = {
    MARTINI_ITP: "https://raw.githubusercontent.com/marrink-lab/martini-forcefields/main/martini_forcefields/regular/v3.0.0/gmx_files/martini_v3.0.0.itp",
    MARTINI_IONS: "https://raw.githubusercontent.com/marrink-lab/martini-forcefields/main/martini_forcefields/regular/v3.0.0/gmx_files/martini_v3.0.0_ions_v1.itp",
    MARTINI_SMALL_MOLS: "https://raw.githubusercontent.com/marrink-lab/martini-forcefields/main/martini_forcefields/regular/v3.0.0/gmx_files/martini_v3.0.0_small_molecules_v1.itp",
    MARTINI_SOLVENTS: "https://raw.githubusercontent.com/marrink-lab/martini-forcefields/main/martini_forcefields/regular/v3.0.0/gmx_files/martini_v3.0.0_solvents_v1.itp",
}

for fname, url in DOWNLOADS.items():
    download_file(url, fname)

This should have downloaded several files to your workspace.

We also need to make a `watermodels.dat` file in the `martini.ff` subdirectory.


In [20]:
WATERMODEL_DAT = MARTINI_FF / "watermodels.dat"

WATERMODEL_DAT.write_text("martini-water\tMARTINI-WATER\tMartini default water model.")

56

Next, we set up the files for the Martini 3.0 forcefield.

In [21]:
from mdpow.forcefields import Forcefield, GromacsSolventModel

MARTINI = Forcefield(
    "Martini",
    solvent_models={
        "octanol": GromacsSolventModel(
            identifier="octanol",
            itp=MARTINI_SOLVENTS.absolute(),
            coordinates=MARTINI_OCTANOL.absolute(),
            forcefield="Martini",
        ),
    },
    forcefield_dir=MARTINI_FF.absolute(),
    ions_itp=MARTINI_IONS.absolute(),
    default_water_itp=MARTINI_SOLVENTS.absolute(),
    default_water_model="martini-water",
    water_models={
        "martini-water": GromacsSolventModel(
            identifier="martini-water",
            itp=MARTINI_SOLVENTS.absolute(),
            coordinates=str(MARTINI_WATER.absolute()),
            forcefield="Martini",
        ),
    },
)

In [22]:
from dataclasses import asdict

for solvent_name, solvent_model in MARTINI.solvent_models.items():
    print(solvent_name)
    print(asdict(solvent_model))

octanol
{'identifier': 'octanol', 'name': 'OCTANOL', 'itp': PosixPath('/home/awsm/MDPOW/doc/examples/martini/martini.ff/martini_v3.0.0_solvents_v1.itp'), 'coordinates': PosixPath('/home/awsm/MDPOW/doc/examples/martini/octanol.gro'), 'description': None, 'forcefield': 'Martini'}


In [30]:
from mdpow.equil import WaterSimulation

sim = WaterSimulation(
    molecule="BENZ",
    ff_class=MARTINI,
    mdp={
        "energy_minimize": str(EM_FILE.absolute()),
        "MD_relaxed": str(EQ_FILE.absolute()),
        "MD_NPT": str(RUN_FILE.absolute()),
        "MD_restrained": str(RUN_FILE.absolute()),
    },
)
sim.topology(str(BENZENE_ITP))
sim.solvate(struct=MARTINI_BENZENE, maxwarn=1)
sim.energy_minimize(maxwarn=1)
sim.MD_relaxed(runtime=5e3)  # should be at least 1e3 ps for production not just 5 ps

mdpow.equil : INFO     [/home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top] Created topology 'system.top' that includes 'martini_v3.0.0_small_molecules_v1.itp'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation] Solvating with water '/home/awsm/MDPOW/doc/examples/martini/water.gro'...
                     :-) GROMACS - gmx editconf, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation
Command line:
  gmx editconf -f /home/awsm/MDPOW/doc/examples/martini/benzene.pdb -o boxed.gro -bt dodecahedron -d 1.0


Back Off! I just backed up boxed.gro to ./#boxed.gro.13#

GROMACS reminds you: "That Was Really Cool" (Butthead)

                     :-) GROMACS - gmx solvate, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation
C

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 3 atoms
Volume: 4050 nm^3, corresponds to roughly 1822500 electrons
No velocities found
    system size :  0.210  0.160  0.248 (nm)
    diameter    :  0.270               (nm)
    center      :  2.987  0.606  2.315 (nm)
    box vectors : 15.000 15.000 18.000 (nm)
    box angles  :  90.00  90.00  90.00 (degrees)
    box volume  :4050.00               (nm^3)
    shift       : -1.284  1.097 -1.512 (nm)
new center      :  1.703  1.703  0.803 (nm)
new box vectors :  2.270  2.270  2.270 (nm)
new box angles  :  60.00  60.00  90.00 (degrees)
new box volume  :   8.27               (nm^3)

         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functiona

gromacs.cbook: INFO     system total charge qtot = 0
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation] After solvation: total charge qtot = 0 = 0
gromacs.cbook: INFO     system total charge qtot = 0
gromacs.setup: INFO     Building the main index file 'main.ndx'...
                     :-) GROMACS - gmx make_ndx, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation
Command line:
  gmx make_ndx -f ionized.tpr -o main.ndx


Reading structure file
Reading file ionized.tpr, VERSION 2023.2 (single precision)
Reading file ionized.tpr, VERSION 2023.2 (single precision)

Back Off! I just backed up main.ndx to ./#main.ndx.26#

GROMACS reminds you: "It Just Tastes Better" (Burger King)

                     :-) GROMACS - gmx make_ndx, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/awsm

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for centering
Selected 4: '__main__'
Select group for output
Selected 0: 'System'


atom name 46 in /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top and /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/solvated.gro does not match (W - W 1)

atom name 47 in /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top and /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/solvated.gro does not match (W - W 1)

atom name 48 in /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top and /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/solvated.gro does not match (W - W 1)

atom name 49 in /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top and /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/solvated.gro does not match (W - W 1)

atom name 50 in /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top and /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/solvated.gro does not match (W - W 1)

atom name 51 in /hom

...

atom name 46 in /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top and /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/solvated.gro does not match (W - W 1)

atom name 47 in /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top and /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/solvated.gro does not match (W - W 1)

atom name 48 in /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top and /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/solvated.gro does not match (W - W 1)

atom name 49 in /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top and /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/solvated.gro does not match (W - W 1)

atom name 50 in /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top and /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/solvated.gro does not match (W - W 1)

atom name 51 in /hom

Setting the LD random seed to -219464132

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'BENZ'

Excluding 1 bonded neighbours molecule type 'W'

atom name 46 in /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top and /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/solvated.gro does not match (W - W 1)

atom name 47 in /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top and /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/solvated.gro does not match (W - W 1)

atom name 48 in /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top and /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/solvated.gro does not match (W - W 1)

atom name 49 in /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top and /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/solvated.gro does not match (W - W 1)

atom name 


Back Off! I just backed up em.log to ./#em.log.3#
Reading file em.tpr, VERSION 2023.2 (single precision)

NOTE: Parallelization is limited by the small number of atoms,
      only starting 1 thread-MPI ranks.
      You can use the -nt and/or -ntmpi option to optimize the number of threads.

1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the CPU
Using 1 MPI thread
Using 4 OpenMP threads 


Back Off! I just backed up em.trr to ./#em.trr.3#

Back Off! I just backed up em.edr to ./#em.edr.3#

Steepest Descents:
   Tolerance (Fmax)   =  1.00000e+01
   Number of steps    =         1000
Step=    0, Dmax= 1.0e-02 nm, Epot=  1.23531e+06 Fmax= 1.48149e+07, atom= 4
Step=    1, Dmax= 1.0e-02 nm, Epot=  7.70813e+05 Fmax= 5.61453e+06, atom= 86
Step=    2, Dmax= 1.2e-02 nm, Epot=  4.81154e+05 Fmax= 2.27826e+06, atom= 86
Step=    

Setting the LD random seed to 1873764311

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'BENZ'

Excluding 1 bonded neighbours molecule type 'W'

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K

Calculated rlist for 1x1 atom pair-list as 1.100 nm, buffer size 0.000 nm

Set rlist, assuming 4x4 atom pair-list, to 1.100 nm, buffer size 0.000 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup

This run will generate roughly 224 Mb of data


{'struct': '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/MD_relaxed/md.gro',
 'top': '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top',
 'ndx': '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/main.ndx',
 'qscript': ['./local.sh'],
 'mainselection': None,
 'deffnm': 'md',
 'includes': ['/home/awsm/MDPOW/mdpow/top',
  '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top']}

In [41]:
%%script bash
MD_PATH=./Equilibrium/water/MD_relaxed
chmod +x $MD_PATH/local.sh
$MD_PATH/local.sh >> mdrun.log

CalledProcessError: Command 'b'MD_PATH=./Equilibrium/water/MD_relaxed\nchmod +x $MD_PATH/local.sh\n$MD_PATH/local.sh >> mdrun.log\n'' returned non-zero exit status 66.

In [ ]:

# run simulation externally or use MDrunner
# (see docs for using mpi etc)
import gromacs

r = gromacs.run.MDrunner(
    dirname=sim.dirs["MD_relaxed"],
    deffnm="md",
    c="md.pdb",
    cpi=True,
    append=True,
    v=True,
)
r.run()  # runs mdrun in the python shell


sim.MD(
    runtime=10, qscript=["local.sh"]
)  # should be at least 10e3 ps for production, not just 10 ps
# run simulation
r = gromacs.run.MDrunner(
    dirname=sim.dirs["MD_NPT"], deffnm="md", c="md.pdb", cpi=True, append=True, v=True
)
r.run()  # runs mdrun in the python shell


import mdpow.fep

gwat = mdpow.fep.Ghyd(simulation=sim, runtime=10)
gwat.setup()

# run multiple simulations on cluster


O = mdpow.equil.OctanolSimulation(molecule="BNZ")
O.topology("benzene.itp")
O.solvate(struct="benzene.pdb")
O.energy_minimize()
O.MD_relaxed(runtime=0.5)